# Starting from replicates, deriving from each replicate of the mapping the difference between research and burden within diseases

In [12]:
library(data.table)

In [25]:
GBD <- read.table("../Data/DALY_YLL_deaths_per_region_and_27_diseases_2005.txt")
GBD <- GBD[order(GBD$Region,GBD$Disease),]

In [13]:
Mgbd <- read.table("../Data/27_gbd_groups.txt")
sms <- list.files("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/")
dis <- as.numeric(substr(sms,25,nchar(sms)-4))

In [31]:
k <- 1
DF <- fread(paste(c("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/Metrics_over_replicates_",
                    as.character(k),".txt"),collapse=""))
regs <- sort(unique(DF$Region))
regs <- regs[regs!="All" & regs!="Non-HI"]

In [89]:
data_f <- data.frame()

In [96]:
for(k in dis[dis!=0]){

    DF <- fread(paste(c("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/Metrics_over_replicates_",
                    as.character(k),".txt"),collapse=""))

    
    DFr <- DF[DF$Region%in%regs & DF$Dis == "dis",]
    DFr$RCTs_all <- rep(DF$RCTs[DF$Dis=="dis" & DF$Region=="All"],each=length(regs))
    DFr$RCTs_NHI <- rep(DF$RCTs[DF$Dis=="dis" & DF$Region=="Non-HI"],each=length(regs))
    DFr$Patients_all <- rep(DF$Patients[DF$Dis=="dis" & DF$Region=="All"],each=length(regs))
    DFr$Patients_NHI <- rep(DF$Patients[DF$Dis=="dis" & DF$Region=="Non-HI"],each=length(regs))


    nb_sims <- nrow(DFr)/length(regs)

    DFr$RCTs_prop_all <- 100*DFr$RCTs/DFr$RCTs_all
    DFr$RCTs_prop_NHI <- 100*DFr$RCTs/DFr$RCTs_NHI
    DFr$Patients_prop_all <- 100*DFr$Patients/DFr$Patients_all
    DFr$Patients_prop_NHI <- 100*DFr$Patients/DFr$Patients_NHI

    gbd_mt <- GBD[GBD$Disease==as.character(Mgbd$x[k]) & GBD$metr=="daly" & GBD$Region!="All",]
    gbd_mt$burden_prop_all <- 100*gbd_mt$burden/sum(gbd_mt$burden)
    gbd_mt$burden_prop_NHI <- 100*gbd_mt$burden/sum(gbd_mt$burden[gbd_mt$Region!="High-income"])
    DFr$burden_daly_prop_all <- rep(gbd_mt$burden_prop_all,times=nb_sims)
    DFr$burden_daly_prop_NHI <- rep(gbd_mt$burden_prop_NHI,times=nb_sims)

    gbd_mt <- GBD[GBD$Disease==as.character(Mgbd$x[k]) & GBD$metr=="death" & GBD$Region!="All",]
    gbd_mt$burden_prop_all <- 100*gbd_mt$burden/sum(gbd_mt$burden)
    gbd_mt$burden_prop_NHI <- 100*gbd_mt$burden/sum(gbd_mt$burden[gbd_mt$Region!="High-income"])
    DFr$burden_death_prop_all <- rep(gbd_mt$burden_prop_all,times=nb_sims)
    DFr$burden_death_prop_NHI <- rep(gbd_mt$burden_prop_NHI,times=nb_sims)

    gbd_mt <- GBD[GBD$Disease==as.character(Mgbd$x[k]) & GBD$metr=="yld" & GBD$Region!="All",]
    gbd_mt$burden_prop_all <- 100*gbd_mt$burden/sum(gbd_mt$burden)
    gbd_mt$burden_prop_NHI <- 100*gbd_mt$burden/sum(gbd_mt$burden[gbd_mt$Region!="High-income"])
    DFr$burden_yld_prop_all <- rep(gbd_mt$burden_prop_all,times=nb_sims)
    DFr$burden_yld_prop_NHI <- rep(gbd_mt$burden_prop_NHI,times=nb_sims)

    gbd_mt <- GBD[GBD$Disease==as.character(Mgbd$x[k]) & GBD$metr=="yll" & GBD$Region!="All",]
    gbd_mt$burden_prop_all <- 100*gbd_mt$burden/sum(gbd_mt$burden)
    gbd_mt$burden_prop_NHI <- 100*gbd_mt$burden/sum(gbd_mt$burden[gbd_mt$Region!="High-income"])
    DFr$burden_yll_prop_all <- rep(gbd_mt$burden_prop_all,times=nb_sims)
    DFr$burden_yll_prop_NHI <- rep(gbd_mt$burden_prop_NHI,times=nb_sims)

    DFr$sim <- rep(1:nb_sims,each=length(regs))

    df <- 
    DFr[,.(
           #DALYs
           #RCTs all vs burden all
           sum(abs(RCTs_prop_all-burden_daly_prop_all)),
           #Patients all vs burden all
           sum(abs(Patients_prop_all-burden_daly_prop_all)),
           #RCTs NHI vs burden NHI
           sum(abs(RCTs_prop_NHI-burden_daly_prop_NHI)),
           #Patients NHI vs burden NHI
           sum(abs(Patients_prop_NHI-burden_daly_prop_NHI)),
           #Deaths
           sum(abs(RCTs_prop_all-burden_death_prop_all)),
           sum(abs(Patients_prop_all-burden_death_prop_all)),
           sum(abs(RCTs_prop_NHI-burden_death_prop_NHI)),
           sum(abs(Patients_prop_NHI-burden_death_prop_NHI)),
           #YLD
           sum(abs(RCTs_prop_all-burden_yld_prop_all)),
           sum(abs(Patients_prop_all-burden_yld_prop_all)),
           sum(abs(RCTs_prop_NHI-burden_yld_prop_NHI)),
           sum(abs(Patients_prop_NHI-burden_yld_prop_NHI)),
           #YLL
           sum(abs(RCTs_prop_all-burden_yll_prop_all)),
           sum(abs(Patients_prop_all-burden_yll_prop_all)),
           sum(abs(RCTs_prop_NHI-burden_yll_prop_NHI)),
           sum(abs(Patients_prop_NHI-burden_yll_prop_NHI))),by="sim"]


    dui <- df[,lapply(.SD,function(x){quantile(x,probs=c(0.025,0.5,0.975),na.rm=TRUE)}),.SDcols=paste("V",1:16,sep="")]

    dui <- cbind(as.character(Mgbd$x[k]),c("low","med","up"),dui)
    names(dui)<-c("Disease","UI",
    paste(rep(c("RCTs_vs","Patients_vs"),times=8),      
          rep(c("daly","death","yld","yll"),each=4),
          rep(rep(c("all","NHI"),each=2),times=4),sep="_"))
    

    data_f <- rbind(data_f,dui)
    
    }

,Disease,UI,RCTs_vs_daly_all,Patients_vs_daly_all,RCTs_vs_daly_NHI,Patients_vs_daly_NHI,RCTs_vs_death_all,Patients_vs_death_all,RCTs_vs_death_NHI,Patients_vs_death_NHI,RCTs_vs_yld_all,Patients_vs_yld_all,RCTs_vs_yld_NHI,Patients_vs_yld_NHI,RCTs_vs_yll_all,Patients_vs_yll_all,RCTs_vs_yll_NHI,Patients_vs_yll_NHI
1,Tuberculosis,low,80.0076364874604,93.3580882563697,97.2479247656515,94.9993017954324,87.7586709194589,96.8678443850018,104.280691962905,98.6878488537501,86.1668082413071,101.756867344985,99.5328261716977,104.11639697178,79.5367149565251,92.5394370677308,97.4548372581497,94.1370351608974
2,Tuberculosis,med,89.010579125441,108.47907746138,113.162360218723,109.434292271281,96.9014915676412,114.187709215418,120.09720030231,115.454849813439,94.5859684491647,130.224744125344,115.158176231574,132.587041875363,88.5303218713028,105.790699432469,113.639972268861,106.554861962721
3,Tuberculosis,up,99.9053896035274,144.039557284296,141.002287796366,145.522059505732,107.064861523491,149.86177261528,147.521639625483,151.591410530616,104.942759461256,166.126038929894,142.027203726307,168.656429914583,100.138848930642,141.272894955593,141.528728218644,142.637387538817
4,Hepatitis,low,159.868713678244,120.69510978506,288.411802871978,152.457520342732,151.111304033973,116.052537626882,279.434140812074,149.521545946932,119.537436427682,68.3872149059026,219.552171291141,99.0545236392442,162.741384490153,123.447926556734,291.24225201552,155.062197427528
5,Hepatitis,med,163.969932324367,148.432627748347,301.760448485101,214.55145427874,154.751896965432,140.724066852732,292.782786425197,206.897114418435,123.773417775565,93.7169271274629,232.900816904264,154.193586436456,166.859819607195,150.960051372988,304.590897628643,217.21990108138
6,Hepatitis,up,167.545781259824,153.353977422869,318.848823488462,312.12720191392,158.268718069211,146.39047227087,309.871161428559,303.159865297891,128.180934829064,123.728748500565,249.989191907625,246.883753103251,170.435668542652,155.87900454154,321.679272632005,314.957651057462
7,Neoplasms,low,89.0295817004856,108.071151001853,253.223771062055,357.31286219492,76.083927117562,94.0126364231995,233.875277084511,339.904163462827,46.3105618660379,62.5289729732401,189.045766950365,300.490593354893,90.0191564106632,109.114453524722,254.231968167574,358.222737408869
8,Neoplasms,med,89.7602802735402,124.243307590815,257.200375897144,643.397278738234,76.8146256906167,110.098045683023,237.854051292463,625.046172599518,46.9709326914235,78.3071846685661,193.003942521395,585.776909955058,90.7498549837179,125.287160179065,258.208573002664,644.307153952184
9,Neoplasms,up,90.5127238794352,129.330613855478,261.372659374776,813.024140774372,77.5670692965116,115.183290217272,242.011240588342,794.874400429018,47.6379345157534,83.4283513832927,197.153045325147,753.081088239273,91.5022985896128,130.374466443728,262.380856480296,813.939355966955
10,Cardiovascular and circulatory diseases,low,109.376721777827,89.0673965542044,252.0617053478,129.128611868746,99.6053389402941,78.0606315353576,240.098261010186,133.597912585048,86.9429892764762,62.8941306637614,238.420102985018,124.572222480464,111.019330303539,90.9379336098141,253.228318093833,128.273445973808
